#Problem 1

To prove the central difference relation, we express $y(x_0+h)$ and $y(x_0-h)$ with Taylor series for small $h$.

\begin{equation}
y_1 = y(x_0+h) = y_0 + y_0'h+\frac{1}{2}y_0''h^2+\frac{1}{6}y_0'''h^3+\frac{1}{24}y_0^{(4)}h^4+O(h^5) \\
y_{-1} = y(x_0-h) = y_0 - y_0'h+\frac{1}{2}y_0''h^2-\frac{1}{6}y_0'''h^3+\frac{1}{24}y_0^{(4)}h^4-O(h^5)
\end{equation}

If we sum these two:

\begin{equation}
y_1+y_{-1} \approx 2y_0 + y_0''h^2 + \frac{1}{12}y^{(4)}h^4 \\
y_0'' \approx \frac{y_1 - 2y_0 +y_{-1}}{h^2} - \frac{1}{12}y^{(4)}
\end{equation}

Thus, we found the error term $O(h^2) = -\frac{1}{12}y^{(4)}(\xi)$.

#Problem 2

In this problem we similarly expand also $y_2 = y(x_0+2h)$ and $y_{-2} = y(x_0-2h)$:

\begin{equation}
y_2 = y(x_0+2h) = y_0 + 2hy_0'+\frac{(2h)^2}{2}y_0''+\frac{(2h)^3}{6}y_0'''+\frac{(2h)^4}{24}y_0^{(4)}+O(h^5) \\
y_{-2} = y(x_0-2h) = y_0 - 2hy_0'+\frac{(2h)^2}{2}y_0''-\frac{(2h)^3}{6}y_0'''+\frac{(2h)^4}{24}y_0^{(4)}-O(h^5)
\end{equation}

It is possible to sum these terms with certain coefficients:

\begin{equation}
-y_2-y_{-2}+16y_1+16y_{-1}-30y_0 = 12h^2y_0''-\frac{2}{15}h^6y_0^{(6)} \\
y_0''=\frac{-y_2+16y_1-30y_0+16y_{-1}-y_{-2}}{12h^2}+\frac{1}{90}h^4y_0^{(6)}
\end{equation}

So $O(h^4) = \frac{1}{90}h^4y_0^{(6)}$.

#Problem 3

For a function $f(x) = \arctan{(x)}$ at $x_0=0$, the first iteration of Richardson extrapolation gives:

\begin{equation}
y_0'=\frac{4}{3}F(h/2) - \frac{1}{3}F(h) - \frac{1}{4}\psi_4h^4-\frac{5}{16}\psi_6h^6
\end{equation}

where:

\begin{equation}
F(h) = \frac{y(x_0+h) - y(x_0 - h)}{2h} \\
\Psi(h) = \frac{h^2}{6}y_0''' + \frac{h^4}{120}y_0^{(5)} + \frac{h^6}{5040}y_0^{(7)} + ... = \psi_2h^2 + \psi_4h^4 + \psi_6h^6 + ...
\end{equation}

We can repeat it to second iteration as:

\begin{equation}
y_0'(h) = \tilde{F}(h)+\tilde{\Psi}(h) \\
\tilde{F}(h) = \frac{4}{3}F(h/2) - \frac{1}{3}F(h) \\
\tilde{\Psi}(h) = \tilde{\psi_4}h^4 + \tilde{\psi_6}h^6 + ... = - \frac{\psi_4}{4}h^4 - \frac{5\psi_6}{16}h^6 + ...\\
\end{equation}

And so:

\begin{equation}
y_0'=\frac{16}{15}\tilde{F}(h/2) - \frac{1}{15}\tilde{F}(h) - \frac{1}{20}\tilde{\psi_6}h^6 - ...
\end{equation}

In [1]:
import numpy as np

In [2]:
def f(x):
  return np.arctan(x)

def df(x):
  return 1/(1 + x**2)

In [3]:
def central_dif(h):
  return (f(x + h) - f(x - h))/(2*h)

def richardson(x, h):
  F_h = central_dif(h)
  F_h_half = central_dif(h/2)
  R_1 = 4*F_h_half/3 - F_h/3

  F_h_quart = central_dif(h/4)
  R_2 = 4*F_h_quart/3 - R_1/3

  ans = 16*R_2/15 - R_1/15

  return ans

In [4]:
x = 0
h = 0.1

richardson_ans = richardson(x, h)

In [5]:
print("Derivative with two iterations Richardson extrapolation = ", richardson_ans)
print("Exact derivative x0 = ", df(0))

Derivative with two iterations Richardson extrapolation =  0.9997059071798116
Exact derivative x0 =  1.0


As we can see, we found the derivative at $x_0=0$ with good accuracy.

To find the derivative with splines, we must make quadratic splines around our point $x_0$. Each spline is defined as:

\begin{equation}
f(x_{i-1}) = a_{i-1}x_{i-1}^2 + b_{i-1}x_{i-1}^2 + c_{i-1} = a_{i}x_{i-1}^2 + b_{i}x_{i-1}^2 + c_{i}
\end{equation}

And the derivatives on nodes must be equal too:

\begin{equation}
2a_{i-1}x_{i-1} + b_{i-1} = 2a_ix_{i-1} + b_i
\end{equation}

These $3n$ coefficients can be found as a solution to linear system of equation $AX=B$, and the derivative at $x_n = x_0$ is $f'(x_0) = a_nx_0+b_n$.

P.S. I am being honest, I am reusing the function for splines that I wrote a long time ago, even long before ChatGPT, and slightly improved it.

In [6]:
def quadratic_spline(x, xVals, yVals):
  n = len(xVals) - 1
  size = 3 * n
  A = np.zeros((size, size))
  B = np.zeros(size)
  i = 0
  for k in range(n):  #finding x_{i}th terms
    A[i, 3*k] = xVals[k]**2
    A[i, 3*k+1] = xVals[k]
    A[i, 3*k+2] = 1
    B[i] = yVals[k]
    i += 1

    A[i, 3*k] = xVals[k+1]**2  #for x_{i+1}
    A[i, 3*k+1] = xVals[k+1]
    A[i, 3*k+2] = 1
    B[i] = yVals[k+1]
    i += 1

  for k in range(1, n):  #for derivatives
    A[i, 3*(k-1)] = 2 * xVals[k]
    A[i, 3*(k-1)+1] = 1
    A[i, 3*k] = -2 * xVals[k]
    A[i, 3*k+1] = -1
    i += 1

  A[i, 0] = 1  #boundary condition for the first segment
  i += 1

  coef = np.linalg.solve(A, B)
  for k in range(n):
    if xVals[k] <= x <= xVals[k+1]:
      a = coef[3*k]
      b = coef[3*k+1]
      c = coef[3*k+2]
      return 2*a*x + b  # Return the derivative at x

In [7]:
xVals = np.linspace(-0.1, 0.1, 10)
yVals = f(xVals)

In [8]:
print("The derivative at x=0 with splines method = ", quadratic_spline(0, xVals, yVals))

The derivative at x=0 with splines method =  0.9999588507846722


Again, we get the answer close to real value.